In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import joblib
emb_matrix_train = joblib.load("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/support_files/sentipol_embedd_7410_128_768_alberto_training.joblib")
wmal_matrix_train = joblib.load("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/support_files/sentipolc_wmal_training_7410_128.joblib")

In [ ]:
import pandas as pd
dataset_DF = pd.read_csv("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/data/sentipolc_train_processed.csv")

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
# define example
data = dataset_DF["opos"]
values = array(data)
print(values)
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print(integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)
# invert first example
inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])
print(inverted)

[0 0 0 ... 0 1 1]
[0 0 0 ... 0 1 1]
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [0. 1.]
 [0. 1.]]
[0]


In [ ]:
from keras.layers import Input, Dense, LSTM, Bidirectional, TimeDistributed, Flatten, Concatenate
!pip install keras-self-attention

In [ ]:
from keras_self_attention import SeqWeightedAttention, SeqSelfAttention

In [ ]:
import keras
input_emb = Input(shape=(128,768))
input_wmal = Input(shape=(128,))

#Bi-LSTM
#bi = Bidirectional(LSTM(32, return_sequences=True))(input_emb)
bi = LSTM(64, return_sequences=True)(input_emb)
#att = SeqWeightedAttention()(bi)
att = SeqSelfAttention(attention_width=15,attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL,
                       attention_activation="sigmoid",
                       kernel_regularizer=keras.regularizers.l2(1e-6),
                       bias_regularizer=keras.regularizers.l1(1e-4),
                       attention_regularizer_weight=1e-4,
                       use_attention_bias=True,return_attention=False) (bi)
flat = Flatten()(att)

#WMAL 
d1 = Dense(64) (input_wmal)

#Concat
concat = Concatenate(axis=1)([flat, d1])

d2 = Dense(32) (concat)
output = Dense(2,activation="softmax") (d2)

m = keras.Model(inputs=[input_emb,input_wmal], outputs=output)
m.compile(loss="categorical_crossentropy",optimizer='adam',metrics=["accuracy"])





In [ ]:
m.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 768)]   0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     (None, 128, 64)      213248      input_1[0][0]                    
__________________________________________________________________________________________________
seq_self_attention (SeqSelfAtte (None, 128, 64)      4097        lstm[0][0]                       
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 128)]        0                                            
_______________________________________________________________________________________

In [ ]:
m.fit([emb_matrix_train,wmal_matrix_train],onehot_encoded,batch_size=128, epochs=5)

Epoch 1/5
58/58 [==============================] - 34s 580ms/step - loss: 0.5179 - accuracy: 0.7622
Epoch 2/5
58/58 [==============================] - 34s 581ms/step - loss: 0.3881 - accuracy: 0.8360
Epoch 3/5
58/58 [==============================] - 34s 589ms/step - loss: 0.2951 - accuracy: 0.8796
Epoch 4/5
58/58 [==============================] - 34s 586ms/step - loss: 0.1826 - accuracy: 0.9328
Epoch 5/5
58/58 [==============================] - 34s 584ms/step - loss: 0.0998 - accuracy: 0.9695


In [ ]:
#emb_matrix_test = joblib.load("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/support_files/creagritrend_embedd_986_128_768_alberto.joblib")
#dataset_test = pd.read_csv("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/data/creagritrend_processed.csv")
#wmal_matrix_test = joblib.load("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/support_files/creagritrend_wmal_agritrend_986_128.joblib")

emb_matrix_test = joblib.load("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/support_files/sentipol_embedd_2000_128_768_alberto_test.joblib")
dataset_test = pd.read_csv("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/data/sentipolc_test_processed.csv")
wmal_matrix_test = joblib.load("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/support_files/sentipolc_wmal_test_2000_128.joblib")

In [ ]:
# define example
data = dataset_test["opos"]
values = array(data)
print(values)
# integer encode
integer_encoded_test = label_encoder.transform(values)
##print(integer_encoded_test)
# binary encode
integer_encoded_test = integer_encoded_test.reshape(len(integer_encoded_test), 1)
onehot_encoded_test = onehot_encoder.transform(integer_encoded_test)
#print(onehot_encoded_test)

[0 1 0 ... 1 0 1]


In [ ]:
predictions = m.predict([emb_matrix_test,wmal_matrix_test])

In [ ]:
print(predictions)

[[9.9976951e-01 2.3054103e-04]
 [9.7447574e-01 2.5524205e-02]
 [9.9775910e-01 2.2408606e-03]
 ...
 [2.8130088e-02 9.7186995e-01]
 [9.9755412e-01 2.4458903e-03]
 [5.9742219e-04 9.9940252e-01]]


In [ ]:
import numpy as np
predictions_vec = np.zeros((2000,4))

In [ ]:
i = 0
confidences= []
for item in predictions:
  index = np.argmax(item);
  confidences.append(item[index])
  predictions_vec[i][index] = 1
  i = i+1

In [ ]:
labels_to_eval = []
for vec in predictions_vec:
  inverted = label_encoder.inverse_transform([argmax(vec)])[0]
  labels_to_eval.append(inverted)

In [ ]:
len(labels_to_eval)

2000

In [ ]:
dataset_test["predicted_label_pos"] = labels_to_eval
dataset_test["confidence_label_pos"] = confidences

In [ ]:
!pip install scikit-learn

from sklearn.metrics import classification_report
print(classification_report(dataset_test["opos"],labels_to_eval,digits=5))

              precision    recall  f1-score   support

           0    0.90821   0.85255   0.87950      1648
           1    0.46358   0.59659   0.52174       352

    accuracy                        0.80750      2000
   macro avg    0.68589   0.72457   0.70062      2000
weighted avg    0.82995   0.80750   0.81653      2000



In [ ]:
dataset_test

,Unnamed: 0,idtwitter,opos,oneg,text,label,clean,predicted_label_pos,confidence_label_pos
0,0,507074506880712705,0,0,Tra 5 minuti presentazione piano scuola del go...,neutral,tra <number> minuti presentazione piano scuola...,0,0.999770
1,1,507075789456961536,1,0,@matteorenzi: Alle 10 appuntamento su http://t...,pos,<user> alle <number> appuntamento su <url> <ha...,0,0.974476
2,2,507077511902425088,0,1,#labuonascuola gli #evangelisti #digitali non ...,neg,<hashtag> la buona scuola </hashtag> gli <hash...,0,0.997759
3,3,507079183315787777,0,0,Riforma scuola Tutto il discorso di Renzi su ...,neutral,riforma scuola tutto il discorso di renzi su o...,0,0.998686
4,4,507080190225563648,0,0,.@matteorenzi @MiurSocial #labuonascuola basta...,neutral,<user> <user> <hashtag> la buona scuola </hash...,1,0.502491
...,...,...,...,...,...,...,...,...,...
1995,1995,645372190645481472,0,0,Anche prodotti alimentari tipici pugliesi in v...,neutral,anche prodotti alimentari tipici pugliesi in v...,1,0.768202
1996,1996,645628412225265664,0,0,intensità di vita https://t.co/jv4aARxzhz,neutral,intensità di vita <url>,0,0.791224
1997,1997,645919232367161344,1,1,Oggi tutti che iniziano l'università e io sul ...,mixed,oggi tutti che iniziano l università e io sul ...,1,0.971870
1998,1998,648567080107790336,0,1,@GliIntoccabili @nonleggerlo Ma Ferrero? il co...,neg,<user> <user> ma ferrero ? il compagno ferrero...,0,0.997554


In [ ]:
#m.save("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/models/model_2_pos_sentipolc_0.70062.h5")

In [ ]:
#dataset_test.to_csv("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/models/predictions_SENTIPOLC_pos_test_model2.csv");